This script loop infinitely to evaluate new rankings added in the db

In [ ]:
# tipi 86
# killbill evaluation ; cd ~/twinews-logs ; jupython -o nohup-evaluation.out --venv st-venv ~/Workspace/Python/Datasets/Twinews/twinews/evaluation/evaluation.ipynb

In [1]:
import os ; os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
isNotebook = '__file__' not in locals()

In [3]:
TEST = isNotebook

In [4]:
from systemtools.hayj import *
from systemtools.location import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from databasetools.mongo import *
from newstools.goodarticle.utils import *
from nlptools.preprocessing import *
from nlptools.news import parser as newsParser
from machinelearning.iterator import *
from twinews.utils import *
from twinews.evaluation import metrics
from twinews.evaluation.utils import *
import time
import pymongo

In [5]:
logger = Logger(tmpDir('logs') + "/evaluation.log") if isNotebook else Logger("evaluation.log")
tt = TicToc(logger=logger)
tt.tic()

--> tictoc starts...


-1

In [6]:
iterations = 1 if isNotebook else 10000000
sleep = 0 if isNotebook else 30
exceptionSleep = 10

In [7]:
metricFuncts = \
{
    'ndcg': metrics.ndcg,
    'ndcg@10': metrics.ndcgAt10,
    'ndcg@100': metrics.ndcgAt100,
    'mrr': metrics.mrr,
    'p@10': metrics.pAt10,
    'p@100': metrics.pAt100,
    'map': metrics.map,
}

In [8]:
if TEST:
    metricFuncts = \
    {
        'ndcg': metrics.ndcg,
        # 'ndcg@10': metrics.ndcgAt10,
        # 'ndcg@100': metrics.ndcgAt100,
        # 'mrr': metrics.mrr,
        # 'p@10': metrics.pAt10,
        # 'p@100': metrics.pAt100,
        # 'map': metrics.map,
    }

In [9]:
# We get scores collection et the rankings GridFS:
twinewsScores = getTwinewsScores(logger=logger)
twinewsRankings = getTwinewsRankings(logger=logger)
# For a certain amount of iterations:
for i in range(iterations):
    # We get all
    modelsKeys = twinewsRankings.keys()
    # For all model instances:
    for modelKey in modelsKeys:
        # We init the eval data to None:
        evalData = None
        rankings = None
        # For all metrics:
        for metricKey, metricFunct in metricFuncts.items():
            # If we didn't added the score previously:
            if not twinewsScores.has({'id': modelKey, 'metric': metricKey}):
                try:
                    # We print infos:
                    log("Computing " + metricKey + " score of " + modelKey + "...", logger)
                    # We get all data:
                    meta = twinewsRankings.getMeta(modelKey)
                    splitVersion = meta['splitVersion']
                    maxUsers = meta['maxUsers']
                    modelName = meta['model']
                    # We get eval data:
                    if evalData is None:
                        log("Downloading eval data version " + str(splitVersion) + "...", logger)
                        evalData = getEvalData(splitVersion, logger=logger, maxExtraNews=0)
                    candidates = evalData['candidates']
                    # We get rankings:
                    if rankings is None:
                        log("Downloading rankings of " + modelKey + "...", logger)
                        rankings = twinewsRankings[modelKey]
                        checkRankings(rankings, candidates, maxUsers=maxUsers)
                        log("Done.", logger)
                    # We convert all in a list of rel vectors:
                    rels = []
                    for userId in rankings:
                        for ranking in rankings[userId]:
                            if isinstance(ranking[0], tuple):
                                ranking = [e[0] for e in ranking]
                            rel = rankingToRelevanceVector(ranking, set(evalData['testUsers'][userId].keys()))
                            rels.append(rel)
                    # We compute all scores:
                    scores = []
                    for rel in rels:
                        scores.append(metricFunct(rel))
                    # We mean all scrores:
                    score = np.mean(scores)
                    # And finally we add the score in the db:
                    if not TEST:
                        addTwinewsScore(modelKey, metricKey, score, verbose=False)
                    # We print result:
                    log(metricKey + " score of " + modelKey + ": " + str(truncateFloat(score, 2)), logger)
                except Exception as e:
                    if isNotebook:
                        raise e
                    else:
                        logException(e, logger)
                        time.sleep(exceptionSleep)
    if sleep > 0:
        log("Sleeping " + str(sleep) + " seconds for the iteration " + str(i) + " on " + str(iterations) + "...", logger)
    time.sleep(sleep)

twinews scores initialised.
Computing ndcg score of BM25-73256...
--> tic: 32.19s | message: Eval data loaded
--> toc total duration: 34.92s | message: Got Twinews evaluation data


AssertionError: 